# Build A Rocket And Launch It

Proceduarlly build and sim a flight. This is my attempt to use the [open aerospace rocket documentation tool](https://open-aerospace.github.io/openrocketdoc/) to describe a rocket and generate JSBSim configuration to simulate its flight.

View the raw jupyter notebook: [rocket.ipynb](https://github.com/natronics/JSBSim-Manager/blob/master/rocket.ipynb)

You can run it yourself by cloning this repo and install requirements:

    $ pip install -r requirements.txt

Then run jupyter to edit/run the document in your browser:

    $ jupyter notebook


## Step 1. Design The Engine

Pick an engine design. Well define it based on a desired Isp, thrust, and burn time.

In [ ]:
import locale
from openrocketdoc import document
from openrocketdoc import writers
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

thrust     = 1555.0  # N
burn_time  =   10.0  # s
isp        =  214.0  # s

# Create an engine document
engine = document.Engine('Engine')

# Set our design
engine.Isp = isp
engine.thrust_avg = thrust
engine.t_burn = burn_time

# Print setup
print("Engine Design parameters:\n")
print("      Input     |   Number  | Units ")
print(" -------------- | --------: | :---- ")

print(" %14s |  %8.1f | %s" % ("Isp", engine.Isp, "s"))
print(" %14s |  %s | %s" % ("Thrust", locale.format("%8.1f", engine.thrust_avg, grouping=True), "N"))
print(" %14s |  %8.1f | %s" % ("Burn Time", engine.t_burn, "s"))


All we need to do is create an openrocketdoc Engine with those basic numbers:

```python
from openrocketdoc import document

engine = document.Engine('Engine')
engine.Isp = 214.0
engine.thrust_avg = 1555.0
engine.t_burn = 10.0
```

Everthing else can be computed from that engine class:

In [ ]:
# The Open Rocket Document can compute useful values based on what we defined above.
print("\nOur computed engine will need %0.1f kg of propellent." % engine.m_prop, )
print("It has a total impulse of %s Ns. That would make it a '%s'(%0.0f%%) class motor." % (
        locale.format("%d", engine.I_total, grouping=True),
        engine.nar_code,
        engine.nar_percent
     ))
print("\nGenerated JSBSim engine document:\n\n```xml")
print(writers.JSBSimEngine.dump(engine))
print("```")

## Step 2. Build The Rocket

Now we know how much propellent, guess the density and come up with some parametric rocket design. If we compute some numbers based on a guess of the density of our propellent, we can build up a full rocket desgin from our `engine`. The only hardcoded magic is a prefered lenght-to-diameter ratio.

In [ ]:
prop_density = 1750      # kg/m3  Roughtly HTPB composite solid density[1]
LD           =   10      # Length to width ratio
Nose_LD      =    5

# [1] http://www.lr.tudelft.nl/en/organisation/departments/space-engineering/space-systems-engineering/expertise-areas/space-propulsion/design-of-elements/rocket-propellants/solids/

print("Rocket Design parameters:\n")
print("          Input         |   Number  | Units ")
print(" ---------------------- | --------: | :---- ")

print(" %22s |  %s | %s" % ("Propellent Density", locale.format("%8.1f", engine.thrust_avg, grouping=True), "kg/m3"))
print(" %22s |  %8.1f | " % ("Motor L/D ratio", 10))
print(" %22s |  %8.1f | " % ("Nosecone L/D ratio", 5))


In [ ]:
from math import pi

# volume of propellent needed
prop_volume = engine.m_prop/prop_density

# Solve for the radius/length of the fuel grain (assume solid, end burning)
engine.diameter = 2*(prop_volume/ (2*LD*pi))**(1/3.0)
engine.length = engine.diameter * LD

# Add a nose
nose = document.Nosecone(
    document.Noseshape.TANGENT_OGIVE,  # Shape
    1.0, # shape_parameter
    1.5, # mass
    engine.diameter * Nose_LD,
    diameter=engine.diameter,
    material_name="Aluminium"
)

# Payload section
payload = document.Bodytube(
    "Payload",  # Name
    2.5,        # mass
    0.33,       # length
    diameter=engine.diameter,
    material_name="Aluminium"
)

# Body section the size of the engine
body = document.Bodytube(
    "Body",  # Name
    1.5,     # mass
    engine.length,
    diameter=engine.diameter,
    material_name="Aluminium"
)

body.components = [engine]

# Rocket:
rocket = document.Rocket("Rocket")
stage0 = document.Stage("Sustainer")
stage0.components = [nose, payload, body]
rocket.stages = [stage0]

# Print:
print("Couputer rocket length: %0.1f meters, diameter: %0.2f mm\n" % ((nose.length + payload.length + body.length), (engine.diameter*1000.0)))
print("Generated diagram of the rocket, with a nosecone, fixed length payload section, and motor:")

from IPython.display import SVG, display
display(SVG(writers.SVG.dump(rocket)))

In [ ]:
print("Generated JSBSim 'Aircraft' document:\n\n```xml")
print(writers.JSBSimAircraft.dump(rocket))
print("```")